Script for creating and training the LSTM network models used to generate classical music.

Matilda Wikström

Jesper Larsson

Links to good stuff!

https://github.com/craffel/pretty-midi

https://towardsdatascience.com/generate-piano-instrumental-music-by-using-deep-learning-80ac35cdbd2e





In [1]:
#Imports
import torch
import pretty_midi
import numpy as np
import unicodedata
import re
import numpy as np
from random import shuffle, seed
import os
import io
import time
from tqdm import tnrange, tqdm_notebook, tqdm
# "Our code"
import preprocessing as pp
#import generatemidi as gm # Broken file, fix
import tokenizer as token
import generatemidi as gen

In [2]:
# Using pretty_midi example
fs=30
dict_note = {}
midi_pretty_format = pretty_midi.PrettyMIDI('test.midi')
piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
piano_roll = piano_midi.get_piano_roll(fs=fs)
dict_note[0] = piano_roll

print(len(piano_roll[0]))

29070


In [3]:
#Download the required data create folder structure
#Make sure we are in the correct folder

assert (os.path.basename(os.getcwd())=='LSTM-MusicGenerator'), "Wrong working dir"

#Download the MAESTRO Dataset
if not os.path.isfile('./maestro-v1.0.0-midi.zip'):
    !wget https://storage.googleapis.com/magentadata/datasets/maestro/v1.0.0/maestro-v1.0.0-midi.zip

#Check if we have extracted the files
if not os.path.isdir('./maestro-v1.0.0-midi') and not os.path.isfile('./maestro-v1.0.0/LICENSE'):
    !unzip maestro-v1.0.0-midi.zip


In [4]:
# Preprocessing
list_all_midi = pp.get_list_midi() # 
sampled_200_midi = list_all_midi[0:100]
batch = 1
start_index = 0
note_tokenizer = token.NoteTokenizer()
import pretty_midi

for i in tqdm_notebook(range(len(sampled_200_midi))):
    dict_time_notes = token.generate_dict_time_notes(sampled_200_midi, batch_song=1, start_index=i, use_tqdm=False, fs=5)
    full_notes = token.process_notes_in_song(dict_time_notes)
    for note in full_notes:
        note_tokenizer.partial_fit(list(note.values()))
note_tokenizer.add_new_note('e') # Add empty notes



In [5]:
#Hyper parameters

#unique_notes+1

In [6]:
# Model
import torch.nn as nn
import torch.nn.functional as F

#Input dimension = 50
#Batch dimension = ? May be one, since we are putting them on 
#Hidden size, try different versions
   
input_size = 50
hidden_size = 20
num_layers = 4
is_bidirectional = True
#dropout_rate = 0.75
dropout_rate = 0
batch_size = 1 #96 ### Should be 2085

seq_len = 50
EPOCHS = 4
BATCH_SONG = 16
BATCH_NNET_SIZE = 96 
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 5


unique_notes = note_tokenizer.unique_word #Used in our output layer to map



if is_bidirectional:
    num_directions = 2
else:
    num_directions = 1

#input = torch.randn(1, batch_size, input_size)




class our_LSTM(torch.nn.Module):
    def __init__(self, h0, c0):
        super().__init__()
        self.hn = h0
        self.cn = c0
        
        self.lstm = torch.nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout_rate, bidirectional=is_bidirectional)
        self.fc = torch.nn.Linear(num_directions * hidden_size, unique_notes)
        self.softmax = torch.nn.Softmax(dim=-1)
        
    def forward(self, inp):
        out, (self.hn, self.cn) = self.lstm(inp, (self.hn, self.cn))
        out = self.fc(out)
        #out = self.softmax(out)
        out = out.permute(0,2,1)
        out = out.view(-1,unique_notes)
        
        return out, self.hn, self.cn
    


In [ ]:
## Training loop
seq_len = 50 # input length here.
EPOCHS = 4
BATCH_SONG = 16
BATCH_NNET_SIZE = 96 # Our sequence length here.
TOTAL_SONGS = len(sampled_200_midi)
FRAME_PER_SECOND = 5
from torch import optim
hn = torch.randn(num_layers*num_directions, batch_size, hidden_size,dtype=torch.float32)
cn = torch.randn(num_layers*num_directions, batch_size, hidden_size,dtype=torch.float32)
model = our_LSTM(hn,cn)

#out, h, c = model(in_tensor, h0, c0) # Expects input with a batch dimension.
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


##Train

for epoch in range(0,5):
    batch_song = BATCH_SONG
    frame_per_second = FRAME_PER_SECOND
    shuffle(sampled_200_midi)
    for i in tqdm_notebook(range(0,len(sampled_200_midi),BATCH_SONG), desc='i'):
        #print("i=:",i)
        inputs_nnet_large, outputs_nnet_large = token.generate_batch_song(
                sampled_200_midi, batch_song, start_index=i, fs=frame_per_second, 
                seq_len=seq_len, use_tqdm=False)

        trans_in = note_tokenizer.transform(inputs_nnet_large)
        trans_out = note_tokenizer.transform(outputs_nnet_large)
        #print("Max of out: ", max(trans_out), "Min of out :", min(trans_out))
        trans_out = trans_out - 1
        counter = 0
        model.hn.detach_() 
        model.cn.detach_()
        for j in tqdm_notebook(range(0,len(trans_in)-BATCH_NNET_SIZE,BATCH_NNET_SIZE), desc='j'):
            counter += 1 
            #print("j=:",j)
            #Want to iterate over the sequence in steps of 96
            input_tensor = trans_in[j:j+BATCH_NNET_SIZE]
            target_tensor = trans_out[j:j+BATCH_NNET_SIZE]
            #print("Max of target tensor: ", max(target_tensor), "Min of target tensor: ", min(target_tensor))

            input_tensor = torch.tensor(input_tensor,dtype=torch.float32)
            input_tensor = input_tensor[None,:,:] # Should be (seq_len, batch, input_size)
            input_tensor = input_tensor.permute(1,0,2)
            #print(input_tensor.shape)
            target_tensor = torch.tensor(target_tensor,dtype=torch.long).view(-1)
            pred, hn, cn = model(input_tensor)
            #print("Pred shape: ", pred.shape)
            #print("Target shape: ", target_tensor.shape)
            loss = loss_fn(pred,target_tensor)
            loss.backward(retain_graph=True)
            #if (counter % 50==0): # Resets the hidden states every 50 iteration to speed up the learning
            #    ;# Should probably be replaced by batch dimension.
            optimizer.step()
            optimizer.zero_grad()
            #print(loss/96)





In [ ]:
def generate_from_random(unique_notes, seq_len=50):
  generate = np.random.randint(0,unique_notes-1,seq_len).tolist()
  return generate
    
def generate_from_one_note(note_tokenizer, new_notes='35'):
  generate = [note_tokenizer.notes_to_index['e'] for i in range(49)]
  generate += [note_tokenizer.notes_to_index[new_notes]]
  return generate

def generate_notes(generate, model, unique_notes, max_generated=1000, seq_len=50):
  for i in tqdm_notebook(range(max_generated), desc='genrt'):
    test_input = np.array([generate])[:,i:i+seq_len]
    test_input = torch.tensor(test_input, dtype=torch.float32)
    test_inputa = test_input[None,:,:]
    #print(test_inputa.shape)
    print(test_inputa)
    predicted_note, _, _ = model(test_inputa)
    #random_note_pred = np.random.choice(unique_notes+1, 1, replace=False, p=predicted_note[0])
    #generate.append(random_note_pred[0])
    #print(predicted_note.shape)
    generate.append(predicted_note.argmax(dim=1).item())
  return generate


def write_midi_file_from_generated(generate, midi_file_name = "result.mid", start_index=0, fs=8, max_generated=1000):
 note_string = [note_tokenizer.index_to_notes[ind_note] for ind_note in generate]
 array_piano_roll = np.zeros((128,max_generated+1), dtype=np.int16)
 for index, note in enumerate(note_string[start_index:]):
   if note == 'e':
     pass
   else:
     splitted_note = note.split(',')
     for j in splitted_note:
       array_piano_roll[int(j),index] = 1
 generate_to_midi = pp.piano_roll_to_pretty_midi(array_piano_roll, fs=fs)
 print("Tempo {}".format(generate_to_midi.estimate_tempo()))
 for note in generate_to_midi.instruments[0].notes:
   note.velocity = 100
 generate_to_midi.write(midi_file_name)


In [ ]:
max_generate = 200
unique_notes = note_tokenizer.unique_word
print(unique_notes)
seq_len=50
generate = generate_from_random(unique_notes-1, seq_len)
print(generate)
generate = generate_notes(generate, model, unique_notes, max_generate, seq_len)
print(generate)
write_midi_file_from_generated(generate, "hello.mid", start_index=seq_len-1, fs=7, max_generated = max_generate)

In [ ]:
max_generate = 300
unique_notes = note_tokenizer.unique_word
seq_len=50
generate = generate_from_one_note(note_tokenizer, '72')
generate = generate_notes(generate, model, unique_notes, max_generate, seq_len)
print(generate)
write_midi_file_from_generated(generate, "one_note.mid", start_index=seq_len-1, fs=8, max_generated = max_generate)